## Activate virtual environment

In [3]:
%%bash 
source ./MedandEECS_env/bin/activate
pip -V

pip 24.0 from /Users/qwer/files/developer/Medicine_and_EECS/MedandEECS_env/lib/python3.12/site-packages/pip (python 3.12)


## Import packages

In [4]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from pydantic import BaseModel


## Initialize client

In [6]:
client  = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Define output format

In [2]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
all.csv

In [7]:
# TODO read all.csv and into dataframe
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
read_cluster('./Data/all.csv')

0       MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
1       MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...
2       IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...
3       IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...
4       GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...
                              ...                        
1125    KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...
1126    JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...
1127    TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...
1128    VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...
1129    MSLN,UPK3B,MUC16,CALB2,KLK11,BICDL1,CHAC1,TGM1...
Name: marker, Length: 1130, dtype: object


,dataset,tissue,marker,manual_annotation,manual_CLname,manual_CLID,manual_broadtype,gpt4aug3_annotation,gpt4aug3_CLname,gpt4aug3_CLID,...,SingleR_annotation,SingleR_CLname,SingleR_CLID,SingleR_broadtype,SingleR_agreement,ScType_annotation,ScType_CLname,ScType_CLID,ScType_broadtype,ScType_agreement
0,Azimuth,PBMC,"MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...",Intermediate B cell,transitional stage B cell,CL:0000818,b cell,B Cells,B cell,CL:0000236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Azimuth,PBMC,"MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...",Memory B cell,memory B cell,CL:0000787,b cell,B Cells,B cell,CL:0000236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Azimuth,PBMC,"IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...",Naive B cell,naive B cell,CL:0000788,b cell,B Cells,B cell,CL:0000236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Azimuth,PBMC,"IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...",Plasmablast,plasmablast,CL:0000980,b cell,Plasma Cells,plasma cell,CL:0000786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Azimuth,PBMC,"GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...",CD4+ Cytotoxic T,"CD4-positive, alpha-beta cytotoxic T cell",CL:0000934,t/nk cell,NK Cells,natural killer cell,CL:0000623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,tabulasapiens,Vasculature,"KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...",nk cell,natural killer cell,CL:0000623,t/nk cell,Natural Killer Cells,natural killer cell,CL:0000623,...,NK_cell,natural killer cell,CL:0000623,t/nk cell,1.0,CD8+ NKT-like cells,NaN,NaN,t/nk cell,0.5
1126,tabulasapiens,Vasculature,"JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...",plasma cell,plasma cell,CL:0000786,b cell,Plasma Cells,plasma cell,CL:0000786,...,B_cell,B cell,CL:0000236,b cell,0.5,Plasma B cells,plasma cell,CL:0000786,b cell,1.0
1127,tabulasapiens,Vasculature,"TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...",lymphatic endothelial cell,endothelial cell of lymphatic vessel,CL:0002138,endothelial cell,Lymphatic Endothelial Cells,endothelial cell of lymphatic vessel,CL:0002138,...,Endothelial_cells,endothelial cell,CL:0000115,endothelial cell,0.5,Lymphatic endothelial cells,endothelial cell of lymphatic vessel,CL:0002138,endothelial cell,1.0
1128,tabulasapiens,Vasculature,"VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...",endothelial cell,endothelial cell,CL:0000115,endothelial cell,Endothelial Cells,endothelial cell,CL:0000115,...,Endothelial_cells,endothelial cell,CL:0000115,endothelial cell,1.0,Endothelial,endothelial cell,CL:0000115,endothelial cell,1.0


## Upload file

In [ ]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

## Send prompt to GPT models

In [ ]:
def annotateCell(cellCluster):
    global client
    tissue = cellCluster['tissue']
    cellMarkers = cellCluster['marker']
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": f"Identify cell types of {tissue} cells using the following markers. Identify one cell type for each row. Only provide the cell type name."},
            {"role": "user", "content": f"{cellMarkers}"}
        ],
        response_format=cellTypeFormat,
    )
    return completion.choices[0].message.parsed
